In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/interbank20/sample_submission.csv
/kaggle/input/interbank20/y_train.csv
/kaggle/input/interbank20/sunat_test.csv
/kaggle/input/interbank20/rcc_test.csv
/kaggle/input/interbank20/se_test.csv
/kaggle/input/interbank20/sunat_train.csv
/kaggle/input/interbank20/se_train.csv
/kaggle/input/interbank20/censo_test.csv
/kaggle/input/interbank20/productos.csv
/kaggle/input/interbank20/censo_train.csv
/kaggle/input/interbank20/rcc_train.csv


## Load RCC

In [2]:
rcc_train = pd.read_csv('/kaggle/input/interbank20/rcc_train.csv')
rcc_test = pd.read_csv('/kaggle/input/interbank20/rcc_test.csv')
y_train = pd.read_csv('/kaggle/input/interbank20/y_train.csv', index_col='key_value')

In [3]:
rcc_test['cod_instit_financiera'].fillna(rcc_test['cod_instit_financiera'].value_counts().index[0], inplace=True)
rcc_test['PRODUCTO'].fillna(rcc_test['PRODUCTO'].value_counts().index[0], inplace=True)

In [4]:
dict_ = dict(zip(list(rcc_train.dtypes[rcc_train.dtypes == int].index), ['int32']*len(rcc_train.dtypes[rcc_train.dtypes == int].index)))
rcc_train = rcc_train.astype(dict_)
rcc_test = rcc_test.astype(dict_)

In [5]:
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda

In [6]:
from tqdm.notebook import tqdm 
import gc
list_rcc_train_agg = []

moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_rcc = {'condicion':['max','min','mean','std','sum'],
           'saldo':['max','min','mean','std','sum'],
           'cod_instit_financiera':['nunique','min','max',moda],
           'PRODUCTO':['nunique','min','max',moda],
           'COD_CLASIFICACION_DEUDOR':['nunique','max','min','sum','mean','std',moda]
          }

In [7]:
for n,i in enumerate(tqdm(sorted(set(rcc_train.codmes),reverse=True))):
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).condicion.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()

        print(f'haciendo agg ')
        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_train_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()
        
rcc_train_ = pd.concat(list_rcc_train_agg, axis=1)
del rcc_train, list_rcc_train_agg

haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESG

In [8]:
from tqdm.notebook import tqdm 
list_rcc_test_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_test.codmes),reverse=True))):
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).condicion.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        print(f'haciendo agg')    
        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_test_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

rcc_test_ = pd.concat(list_rcc_test_agg, axis=1)
del rcc_test, list_rcc_test_agg

haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo 

In [9]:
print(rcc_train_.shape, rcc_test_.shape)
keep_columns = list(set(rcc_train_.columns).intersection(rcc_test_.columns))
rcc_train_ = rcc_train_[keep_columns].copy()
rcc_test_ = rcc_test_[keep_columns].copy()
print(rcc_train_.shape, rcc_test_.shape)

(358487, 441) (396666, 455)
(358487, 441) (396666, 441)


In [15]:
### imputar datos categoricos
var_categ = [i for i in rcc_train_.columns if 'mode' in i]
for i in var_categ:
    rcc_train_[i] = rcc_train_[i].fillna(rcc_train_[i].value_counts().index[0])
    rcc_test_[i] = rcc_test_[i].fillna(rcc_train_[i].value_counts().index[0])

In [19]:
### mean encoding
rcc_train_ = pd.concat([rcc_train_, y_train], axis=1)
for i in var_categ:
    me = rcc_train_.groupby([i])['target'].mean().to_dict() 
    rcc_train_[f'{i}_me'] =  rcc_train_[i].map(me) 
    rcc_test_[f'{i}_me'] =  rcc_test_[i].map(me)
    ### imputar datos faltantes
    rcc_test_[f'{i}_me'] =  rcc_test_[f'{i}_me'].fillna(rcc_train_[f'{i}_me'].median())
    
rcc_train_.drop('target', axis=1, inplace=True)
rcc_train_.drop(var_categ, axis=1, inplace=True)
rcc_test_.drop(var_categ, axis=1, inplace=True)

In [26]:
### imputar datos cauntitativos
var_cuanti = [i for i in rcc_train_.columns if 'std' in i]
for i in var_cuanti:
    rcc_train_[i] = rcc_train_[i].fillna(rcc_train_[i].median())
    rcc_test_[i] = rcc_test_[i].fillna(rcc_train_[i].median())

In [29]:
### unir RCC en la base final
train = rcc_train_.copy()
test = rcc_test_.copy()
del rcc_train_, rcc_test_

## Load SE

In [34]:
se_train = pd.read_csv('/kaggle/input/interbank20/se_train.csv', index_col='key_value')
se_test = pd.read_csv('/kaggle/input/interbank20/se_test.csv', index_col='key_value')
# edad y cod_ubi tienen valores nulos con cerca de 1% cada uno

In [35]:
dict_ = {'sexo':'int32',
 'est_cvl':'int32',
 'sit_lab':'int32',
 'cod_ocu':'int32',
 'ctd_hijos':'int32',
 'flg_sin_email':'int32',
 'ctd_veh':'int32',
 'lgr_vot':'int32',
 'prv':'int32',
 'dto':'int32',
 'rgn':'int32',
 'tip_lvledu':'int32'}
se_train = se_train.astype(dict_)
se_test = se_test.astype(dict_)

In [36]:
### imputar datos faltantes en la base se_[train/test]
se_train['edad'].fillna(se_train['edad'].median(), inplace=True)
se_test['edad'].fillna(se_train['edad'].median(), inplace=True)
se_train['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)
se_test['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)

In [37]:
var_categ=['sit_lab','lgr_vot','sexo','rgn','tip_lvledu','flg_sin_email','prv','est_cvl','cod_ocu','dto']
# var_caunti=['edad','ctd_hijos','ctd_veh','cod_ubi']
se_train['target'] = y_train.loc[se_train.index]
for i in var_categ:
    me = se_train.groupby([i])['target'].mean().to_dict() 
    se_train[f'{i}_me'] =  se_train[i].map(me) 
    se_test[f'{i}_me'] =  se_test[i].map(me)
    ### imputar datos faltantes
    se_test[f'{i}_me'] =  se_test[f'{i}_me'].fillna(se_train[f'{i}_me'].median())
se_train.drop('target', axis=1, inplace=True)
se_train.drop(var_categ, axis=1, inplace=True)
se_test.drop(var_categ, axis=1, inplace=True)

In [38]:
### unir SE en la base final
train = train.join(se_train) 
test = test.join(se_test)
del se_train, se_test

In [39]:
### imputar valores faltantes
var_caunti = ['lgr_vot_me','sit_lab_me','cod_ubi','tip_lvledu_me','prv_me','rgn_me',
 'ctd_hijos','edad','cod_ocu_me','ctd_veh','est_cvl_me','dto_me','flg_sin_email_me','sexo_me']
for i in var_caunti:
    train[i] = train[i].fillna(train[i].median())
    test[i] = test[i].fillna(train[i].median())
gc.collect()

20

## Load Sunat

In [45]:
sunat_train = pd.read_csv('/kaggle/input/interbank20/sunat_train.csv')
sunat_test = pd.read_csv('/kaggle/input/interbank20/sunat_test.csv')

In [46]:
### eliminar registros duplicados
sunat_train.drop_duplicates(inplace=True)
sunat_test.drop_duplicates(inplace=True)
sunat_train.shape, sunat_test.shape

((292479, 18), (318821, 18))

In [47]:
#### eliminando registros con fecalta y fecbaja con valores nulos
mask = ~((sunat_train['fecalta'].astype('str')=='nan') & (sunat_train['fecbaja'].astype('str')=='nan'))
sunat_train = sunat_train[mask].copy()
mask = ~((sunat_test['fecalta'].astype('str')=='nan') & (sunat_test['fecbaja'].astype('str')=='nan'))
sunat_test = sunat_test[mask].copy()
sunat_train.shape, sunat_test.shape

((272028, 18), (293080, 18))

In [48]:
dict_ = dict(zip(list(sunat_train.dtypes[sunat_train.dtypes == int].index), ['int32']*len(sunat_train.dtypes[sunat_train.dtypes == int].index)))
sunat_train = sunat_train.astype(dict_)
sunat_test = sunat_test.astype(dict_)

In [49]:
#### hacer agregaciones
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda
from tqdm.notebook import tqdm 
moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_sunat = {'tipcontribuyente':['nunique',moda],
           'tippersona':['nunique',moda],
           'ciiu':['nunique', moda],
           'ubigeo':['nunique',moda],
           'condiciondomicilio':['nunique',moda],
           'estadocontribuyente':['nunique',moda],
           'codvia':['nunique',moda],
           'codzona':['nunique',moda],
           'contabilidad':['nunique',moda],
           'facturacion':['nunique',moda],
           'domiciliado':['nunique',moda],
           'comercioexterior':['nunique',moda],
           'cargorele':['nunique',moda],
           'codentidadtributo':['nunique',moda],
           'estadotributo':['nunique',moda],
           'fecalta':['mean','max', 'nunique'],
           'fecbaja':['mean','max', 'nunique'],
          }

sunat_train_ = sunat_train.groupby('key_value').agg(agg_sunat)
sunat_train_.columns = [i+'_'+j for i,j in sunat_train_.columns]
sunat_test_ = sunat_test.groupby('key_value').agg(agg_sunat)
sunat_test_.columns = [i+'_'+j for i,j in sunat_test_.columns]
del sunat_train, sunat_test

In [51]:
### unir SUNAT en la base final
train = train.join(sunat_train_)
test = test.join(sunat_test_)
del sunat_train_, sunat_test_

In [56]:
test.isna().sum().sort_values().tail(37)

tipo_credito_9_saldo_ult3meses         0
codzona_nunique                   167754
fecalta_nunique                   167754
fecalta_max                       167754
fecalta_mean                      167754
estadotributo_nunique             167754
codentidadtributo_nunique         167754
cargorele_nunique                 167754
comercioexterior_nunique          167754
domiciliado_nunique               167754
facturacion_nunique               167754
contabilidad_nunique              167754
codvia_nunique                    167754
ciiu_nunique                      167754
fecbaja_nunique                   167754
estadocontribuyente_nunique       167754
tipcontribuyente_nunique          167754
condiciondomicilio_nunique        167754
ubigeo_nunique                    167754
tippersona_nunique                167754
condiciondomicilio_mode           171359
estadotributo_mode                175142
codentidadtributo_mode            176291
facturacion_mode                  176299
ubigeo_mode     

In [ ]:
# var_categ = [i for i in sunat_train_.columns if 'mode' in i]
# for i in var_categ:
#     sunat_train_[i] = sunat_train_[i].fillna(sunat_train_[i].value_counts().index[0])
#     sunat_test_[i] = sunat_test_[i].fillna(sunat_train_[i].value_counts().index[0])

In [ ]:
# sunat_train_['target'] = y_train.loc[sunat_train_.index]
# for i in var_categ:
#     me = sunat_train_.groupby([i])['target'].mean().to_dict() 
#     sunat_train_[f'{i}_me'] =  sunat_train_[i].map(me) 
#     sunat_test_[f'{i}_me'] =  sunat_test_[i].map(me)
#     ### imputar datos faltantes
#     sunat_test_[f'{i}_me'] =  sunat_test_[f'{i}_me'].fillna(sunat_train_[f'{i}_me'].median())
# sunat_train_.drop('target', axis=1, inplace=True)
# sunat_train_.drop(var_categ, axis=1, inplace=True)
# sunat_test_.drop(var_categ, axis=1, inplace=True)

In [ ]:
# var_aux = list(train.isna().sum().sort_values().tail(34).index)
# var_categ = [i for i in var_aux if 'mode' in i]
# var_cuanti = [i for i in var_aux if 'mode' not in i]
# len(var_aux), len(var_categ), len(var_cuanti)

# for i in var_cuanti:
#     train[i] = train[i].fillna(train[i].median())
#     test[i] = test[i].fillna(train[i].median())
# for i in var_categ:
#     train[i] = train[i].fillna(train[i].value_counts().index[0])
#     test[i] = test[i].fillna(train[i].value_counts().index[0])

In [ ]:
# var_cuanti = ['fecbaja_nunique', 'tipcontribuyente_mode_me', 'tippersona_mode_me', 'ciiu_mode_me',
# 'ubigeo_mode_me', 'condiciondomicilio_mode_me', 'estadocontribuyente_mode_me', 
# 'contabilidad_mode_me','codzona_mode_me', 'facturacion_mode_me', 'domiciliado_mode_me', 
# 'comercioexterior_mode_me', 'fecbaja_sum', 'cargorele_mode_me', 'codvia_mode_me', 
# 'fecalta_nunique', 'codvia_nunique', 'estadotributo_nunique', 'codentidadtributo_mode_me', 
# 'tipcontribuyente_nunique','tippersona_nunique', 'ciiu_nunique', 'ubigeo_nunique',
# 'condiciondomicilio_nunique','fecalta_sum', 'estadocontribuyente_nunique',
# 'contabilidad_nunique', 'facturacion_nunique','domiciliado_nunique', 'comercioexterior_nunique',
# 'cargorele_nunique', 'codentidadtributo_nunique', 'codzona_nunique', 'estadotributo_mode_me']
# for i in var_cuanti:
#     train[i] = train[i].fillna(train[i].median())
#     test[i] = test[i].fillna(train[i].median())

## Training

In [57]:
train.shape, test.shape

((358487, 491), (396666, 491))

In [58]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import numpy as np
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]

    learner = CatBoostClassifier(n_estimators=1000,  eval_metric = 'AUC', max_depth = 6)
    learner.fit(Xt, yt,  early_stopping_rounds=10, 
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
    gc.collect()
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)
print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))

********** 0 **********
Learning rate set to 0.128212
0:	test: 0.7496183	test1: 0.7513445	best: 0.7513445 (0)	total: 347ms	remaining: 5m 46s
50:	test: 0.8284170	test1: 0.8264428	best: 0.8264428 (50)	total: 19s	remaining: 5m 53s
100:	test: 0.8374401	test1: 0.8342903	best: 0.8342903 (100)	total: 36.5s	remaining: 5m 24s
150:	test: 0.8439740	test1: 0.8389398	best: 0.8389398 (150)	total: 54.6s	remaining: 5m 7s
200:	test: 0.8488771	test1: 0.8416670	best: 0.8416791 (199)	total: 1m 12s	remaining: 4m 48s
250:	test: 0.8526939	test1: 0.8434047	best: 0.8434047 (250)	total: 1m 29s	remaining: 4m 26s
300:	test: 0.8562642	test1: 0.8448590	best: 0.8448590 (300)	total: 1m 47s	remaining: 4m 9s
350:	test: 0.8591675	test1: 0.8455847	best: 0.8455847 (350)	total: 2m 4s	remaining: 3m 51s
400:	test: 0.8617006	test1: 0.8463838	best: 0.8463838 (400)	total: 2m 21s	remaining: 3m 31s
450:	test: 0.8645468	test1: 0.8470001	best: 0.8470001 (450)	total: 2m 39s	remaining: 3m 13s
500:	test: 0.8668474	test1: 0.8475372	bes

In [ ]:
### solo considerando RCC con 441 variables
# roc auc estimado:  0.8310294226591814
# roc auc varianza:  0.0006799948117676291

### considerando RCC y SE con 455 variables
# roc auc estimado:  0.8449844139781708
# roc auc varianza:  0.0007870894926133227

### considerano RCC, SE y SUNAT con 491 variables
# roc auc estimado:  0.8461160827073155
# roc auc varianza:  0.0009148252331856493

In [59]:
fi.sort_values(ascending=False).head(50)

dto_me                                            0.053906
COD_CLASIFICACION_DEUDOR_mean_ult1mes             0.042380
cod_instit_financiera_mode_ult1mes_me             0.025525
tip_lvledu_me                                     0.025430
RIESGO_DIRECTO_1_saldo_ult1meses                  0.020215
RIESGO_DIRECTO_-1_saldo_ult12meses                0.019466
tipo_credito_11_saldo_ult1meses                   0.019382
edad                                              0.019258
tipo_credito_11_saldo_ult12meses                  0.014625
cod_instit_financiera_mode_ult12mes_me            0.013733
flg_sin_email_me                                  0.013709
COD_CLASIFICACION_DEUDOR_max_ult1mes              0.013379
COD_CLASIFICACION_DEUDOR_0_saldo_ult1meses        0.013341
saldo_sum_ult1mes                                 0.013200
sexo_me                                           0.012886
saldo_mean_ult12mes                               0.011806
saldo_mean_ult1mes                                0.0116

In [60]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import numpy as np
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]

    learner = LGBMClassifier(n_estimators=1000, max_depth = 6, boosting_type='gbdt', min_child_samples=1000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)
print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))  

********** 0 **********


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.8384	training's binary_logloss: 0.305965	valid_1's auc: 0.832376	valid_1's binary_logloss: 0.310822
[100]	training's auc: 0.84982	training's binary_logloss: 0.29713	valid_1's auc: 0.840694	valid_1's binary_logloss: 0.304573
[150]	training's auc: 0.856173	training's binary_logloss: 0.29208	valid_1's auc: 0.843851	valid_1's binary_logloss: 0.301898
[200]	training's auc: 0.860553	training's binary_logloss: 0.288668	valid_1's auc: 0.845601	valid_1's binary_logloss: 0.300595
[250]	training's auc: 0.865097	training's binary_logloss: 0.285162	valid_1's auc: 0.847032	valid_1's binary_logloss: 0.299446
[300]	training's auc: 0.868292	training's binary_logloss: 0.282581	valid_1's auc: 0.848005	valid_1's binary_logloss: 0.298735
[350]	training's auc: 0.871304	training's binary_logloss: 0.280164	valid_1's auc: 0.848654	valid_1's binary_logloss: 0.298166
Early stopping, best iteration is:
[361]	training's auc: 0.8720

In [ ]:
# roc auc estimado:  0.8473652017451274
# roc auc varianza:  0.0007348784830705827

In [61]:
fi.sort_values(ascending=False).head(50)

dto_me                                            0.028081
edad                                              0.021397
saldo_mean_ult12mes                               0.020162
RIESGO_DIRECTO_1_saldo_ult1meses                  0.017712
RIESGO_DIRECTO_-1_saldo_ult12meses                0.017282
cod_ubi                                           0.016960
tipo_credito_11_saldo_ult1meses                   0.015966
RIESGO_DIRECTO_1_saldo_ult12meses                 0.015483
tipo_credito_11_saldo_ult12meses                  0.014953
cod_instit_financiera_max_ult1mes                 0.012837
cod_instit_financiera_mode_ult12mes_me            0.012836
ubigeo_mode                                       0.012634
RIESGO_DIRECTO_-1_saldo_ult1meses                 0.012596
cod_instit_financiera_mode_ult1mes_me             0.012572
tipo_credito_12_saldo_ult12meses                  0.012108
saldo_min_ult1mes                                 0.011907
COD_CLASIFICACION_DEUDOR_0_saldo_ult12meses       0.0115

In [69]:
# fecalta_nunique    0.001005
# fecalta_max        0.005195
# fecalta_mean       0.010287
# fecbaja_nunique    0.000349
# fecbaja_max        0.000992
# fecbaja_mean       0.001972

In [70]:
from catboost import CatBoostClassifier
learner_catboost = CatBoostClassifier(n_estimators=1000, eval_metric = 'AUC', max_depth = 6)
learner_catboost.fit(train, y_train,  early_stopping_rounds=10, verbose = 50)

Learning rate set to 0.126968
0:	total: 482ms	remaining: 8m 1s
50:	total: 21.7s	remaining: 6m 43s
100:	total: 41.1s	remaining: 6m 6s
150:	total: 1m	remaining: 5m 37s
200:	total: 1m 20s	remaining: 5m 19s
250:	total: 1m 38s	remaining: 4m 52s
300:	total: 1m 56s	remaining: 4m 30s
350:	total: 2m 15s	remaining: 4m 9s
400:	total: 2m 33s	remaining: 3m 48s
450:	total: 2m 51s	remaining: 3m 29s
500:	total: 3m 9s	remaining: 3m 8s
550:	total: 3m 28s	remaining: 2m 49s
600:	total: 3m 46s	remaining: 2m 30s
650:	total: 4m 5s	remaining: 2m 11s
700:	total: 4m 23s	remaining: 1m 52s
750:	total: 4m 41s	remaining: 1m 33s
800:	total: 4m 59s	remaining: 1m 14s
850:	total: 5m 18s	remaining: 55.7s
900:	total: 5m 36s	remaining: 37s
950:	total: 5m 54s	remaining: 18.3s
999:	total: 6m 12s	remaining: 0us


In [71]:
from lightgbm import LGBMClassifier
learner_lightgbm = LGBMClassifier(n_estimators=1000, max_depth=6, boosting_type='gbdt', min_child_samples=1000)
learner_lightgbm.fit(train, y_train, eval_metric="auc", verbose=50)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LGBMClassifier(max_depth=6, min_child_samples=1000, n_estimators=1000)

In [72]:
test_probs_catboost = pd.Series(learner_catboost.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs_lightgbm = pd.Series(learner_lightgbm.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs_mean = test_probs_catboost*0.5 + test_probs_lightgbm*0.5
test_probs_mean.name = 'target'
test_probs_mean.shape

(396666,)

In [75]:
test_probs_mean.to_csv("test48.csv")

<a href = 'test48.csv'>Download csv</a>